In [1]:
import math
import pandas as pd
from lenskit import topn
from sklearn.metrics import ndcg_score
from statistics import mean, variance
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
preprocessed_dataset_folder = "../preprocessed_dataset"
ratings_df = pd.read_csv(preprocessed_dataset_folder+"/ratings.csv") 
movies_df = pd.read_csv(preprocessed_dataset_folder+"/movies.csv", index_col="item")

In [3]:
def pipeline(groupsize):
    
    users_ratings = ratings_df.groupby(['user']).count()
    selected = users_ratings['rating'] > 200
    selected_users = users_ratings.loc[selected]
    random_selected = selected_users.sample(groupsize) # sample() returns a random row from the dataframe. The returned object is a dataframe with one row. If you pass a number as argument you specify to select more than one row.
    select_column_df = random_selected.reset_index()['user'] # reset_index() create a new index, and the user became a column. Then, we can filter using the column name
    selected_users = select_column_df.values # iloc select by index, since our dataframe only has one row we read it from the index 0
    print("Selected users: " + str(selected_users))

    train_df, test_df = train_test_split(ratings_df, test_size=0.2)

    user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
    recsys = Recommender.adapt(user_user)
    recsys.fit(train_df)

    test_df['predicted_rating'] = recsys.predict(test_df)

    test_df['relevant'] = test_df['rating'].apply(lambda x: 1 if x > 3 else 0)
    test_df['predicted_relevant'] = test_df['predicted_rating'].apply(lambda x: 1 if x > 3 else 0)

    train_df, test_df = train_test_split(ratings_df, test_size=0.2)

    user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
    recsys = Recommender.adapt(user_user)
    recsys.fit(train_df)
    
    group_list = []
    for i in selected_users:
        selected_movies = recsys.recommend(i, 10)
        group_list.append(selected_movies)
        
    group_df = pd.concat(group_list)
    group_df.drop_duplicates(subset=["item"], inplace=True)
    
    lst = []
    for i in selected_users:
        grp = group_df.copy()
        grp["user"] = i
        grp["pred"] = recsys.predict_for_user(i, grp["item"]).values
        lst.append(grp)
        
    final = pd.concat(lst)
    #Here I implement both least misery and additive aggregations strategies
    final.sort_values(by=["pred"], inplace=True, ascending=False)

    gbadd = final.groupby("item", as_index=False)["pred"].sum() #Additive strat
    gbleast = final.groupby("item", as_index=False)["pred"].min() #Least misery strat

    final_recommendationsAdd = gbadd.sort_values(by=["pred"], ascending=False).head(5)
    final_recommendationsAdd
    
    final_recommendationsleast = gbleast.sort_values(by=["pred"], ascending=False).head(5)
    final_recommendationsleast
        
    evaluateAdd = final_recommendationsAdd.copy()
    for i in selected_users:
        evaluateAdd[i] = recsys.predict_for_user(i, final_recommendationsAdd["item"]).values
    evaluateAdd
    # if scores cannot be predicted replace with 0
    evaluateAdd.fillna(0, inplace=True)
    recommended_ordering = pd.DataFrame(evaluateAdd[["item","pred"]])
    recommended_ordering
        
    evaluateleast = final_recommendationsleast.copy()
    for i in selected_users:
        evaluateleast[i] = recsys.predict_for_user(i, final_recommendationsleast["item"]).values
    evaluateleast
    # if scores cannot be predicted replace them with null value
    evaluateleast.fillna(0, inplace=True)
    recommended_orderingleast = pd.DataFrame(evaluateleast[["item","pred"]])
    recommended_orderingleast
    
    
    ndcg_lstAdd = []
    for i in selected_users:
        evaluateAdd.sort_values(by=[i], ascending=False, inplace=True)
        ev = pd.DataFrame(evaluateAdd[["item", i]])
        val = ndcg_score([ev.item], [recommended_ordering.item])
        ndcg_lstAdd.append(val)

    #return mean(ndcg_lstAdd)
    
    
    ndcg_lstleast = []
    for i in selected_users:
        evaluateleast.sort_values(by=[i], ascending=False, inplace=True)
        ev = pd.DataFrame(evaluateleast[["item", i]])
        val = ndcg_score([ev.item], [recommended_orderingleast.item])
        ndcg_lstleast.append(val)

    return mean(ndcg_lstAdd), mean(ndcg_lstleast)
    

        

In [ ]:
gsize2_least = []
gsize2_add = []
for i in range(1000):
    gsize2_least.append(pipeline(4)[0])
    gsize2_add.append(pipeline(4)[1])
    
print(mean(gsize2_add))
print(mean(gsize2_least))

Selected users: [ 57 294 448 160]


BLAS using multiple threads - can cause oversubscription
found 1 potential runtime problems - see https://boi.st/lkpy-perf


Selected users: [560 514  51 226]
Selected users: [414 573 509  45]
Selected users: [382 603  28 305]
Selected users: [580 200  89 132]
Selected users: [381 567 288 292]
Selected users: [387 489 600 199]
Selected users: [477 381 182  91]
Selected users: [199 560 339  66]
Selected users: [477 599 156 391]
Selected users: [599 596 480  64]
Selected users: [ 42 514 298 249]
Selected users: [525 226 599 368]
Selected users: [382 274  57 603]
Selected users: [489 480 294  57]
Selected users: [469 480 156  45]
Selected users: [603 313  19 509]
Selected users: [610 414 381 509]
Selected users: [ 66  45 339 111]
Selected users: [600 514 313 339]
Selected users: [305 428 111 525]
Selected users: [561 567 307 305]
Selected users: [305 597 391 573]
Selected users: [484 477 318 600]
Selected users: [ 91 219 292 156]
Selected users: [177  91 597 275]
Selected users: [ 45 132  91 368]
Selected users: [534  21 368 428]
Selected users: [288 597  91 428]
Selected users: [202  68 307  42]
Selected users